<a href="https://colab.research.google.com/github/Annihilator0/Twitter-Sentiment-Analysis-using-ML/blob/main/Twitter%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing kaggle library
! pip install kaggle

In [ ]:
# configuring the path of kaggle.jason file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

importing twitter sentiment dataset

In [ ]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

In [ ]:
#extracting compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# printing the stopwords in english
print(stopwords.words('english'))

Data Processing

In [ ]:
#loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [ ]:
# checking the number of rows and columns
twitter_data.shape

In [ ]:
#printinfg the first 5 rows of the dataframe
twitter_data.head()

In [ ]:
# naming the columns and reading the datasets again

column_names = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [ ]:
# checking the number of rows and columns again
twitter_data.shape

In [ ]:
#printinfg the first 5 rows of the dataframe again
twitter_data.head()

In [ ]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

In [ ]:
# checking the distribution of target column(how many positives and negatives)
twitter_data['target'].value_counts()

Convert the target "4" to "1"

In [ ]:
twitter_data.replace({'target' :{4:1}}, inplace=True)

In [ ]:
# checking the distribution of target column(how many positives and negatives) again
twitter_data['target'].value_counts()

0--> Negative Tweet

1--> Positive Tweet

**Stemming** -the process of reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ', content)#remove anything that is not an alphabet
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content
  #remove special characters>tolower>split into words>stem if not present in the stopwords>join

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)#50 min to complete
#create a new column and the content of the column is the stemmed content

In [ ]:
twitter_data.head()

In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

In [ ]:
# separating the data and the label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

Splittin the data to training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)
# 0.2 means 20% goes to test data and rest to  training data satisfy=y means to split values equally or balance positives and negatives

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

Converting Textual data to numerical data

In [ ]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

Training the Machine Learning Model(Logistic Reggression Model)

In [ ]:
model = LogisticRegression(max_iter=1000)#maximeme iteration until best accuracy


In [ ]:
model.fit(X_train, Y_train)

**Model Evaluation**

Accuracy Score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [80]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.79871953125


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [79]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.77668125


Model accuracy = 77.66%

Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future predictions

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [78]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Negative')
else:
  print('The news is Positive')

1
[1]
The news is Positive


In [77]:
X_new = X_test[131]
print(Y_test[131])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Negative')
else:
  print('The news is Positive')

1
[1]
The news is Positive
